# Fluxo de Projeto Machine Learning

## Objetivos

- Compreender as etapas de um projeto ML
- Aplicar boas práticas em cada fase
- Conhecer aspectos éticos em dados

## Pré-requisitos

- Lição anterior: Introdução ao ML


In [ ]:
# Importações necessárias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Configurar seeds para reprodutibilidade
np.random.seed(42)

# Configurar estilo dos gráficos
plt.style.use("default")
sns.set_palette("husl")
plt.rcParams["figure.figsize"] = (10, 6)

## 1. Etapas de um Projeto ML

### Fluxo típico:

1. **Definição do problema**
2. **Coleta e exploração de dados**
3. **Preparação dos dados**
4. **Seleção e treinamento do modelo**
5. **Avaliação do modelo**
6. **Deploy e monitoramento**

Vamos ver cada etapa com um exemplo prático:


## 2. Exemplo Prático: Previsão de Preços de Casas

### Etapa 1: Definição do Problema

- **Objetivo**: Prever preços de casas
- **Tipo**: Regressão (valor contínuo)
- **Métrica**: RMSE (Root Mean Squared Error)
- **Stakeholders**: Imobiliária, compradores


### Etapa 2: Coleta e Exploração de Dados


In [ ]:
# Carregar dataset sintético (baseado no conceito do Boston Housing)
# Vamos criar dados sintéticos para evitar problemas éticos do dataset original
np.random.seed(42)

n_samples = 500
data = {
    "rooms": np.random.normal(6, 1, n_samples),  # número de quartos
    "age": np.random.uniform(0, 100, n_samples),  # idade da casa
    "distance": np.random.exponential(3, n_samples),  # distância do centro
    "tax_rate": np.random.uniform(300, 700, n_samples),  # taxa de impostos
    "education": np.random.uniform(10, 20, n_samples),  # índice educacional
}

# Criar target com relação com features
price = (
    data["rooms"] * 8
    + (100 - data["age"]) * 0.3
    + -data["distance"] * 2
    + -data["tax_rate"] * 0.02
    + data["education"] * 1.5
    + np.random.normal(0, 5, n_samples)  # ruído
)

# Garantir preços positivos
price = np.maximum(price, 10)

# Criar DataFrame
df = pd.DataFrame(data)
df["price"] = price

print("Dataset carregado com sucesso!")
print(f"Shape: {df.shape}")
df.head()

In [ ]:
# Análise exploratória
print("=== Informações Gerais ===")
print(df.info())
print("\n=== Estatísticas Descritivas ===")
print(df.describe())

In [ ]:
# Visualizações exploratórias
plt.figure(figsize=(15, 10))

# Distribuição do target
plt.subplot(2, 3, 1)
plt.hist(df["price"], bins=30, alpha=0.7)
plt.xlabel("Preço")
plt.ylabel("Frequência")
plt.title("Distribuição dos Preços")

# Correlação entre features e target
features = ["rooms", "age", "distance", "tax_rate", "education"]
for i, feature in enumerate(features):
    plt.subplot(2, 3, i + 2)
    plt.scatter(df[feature], df["price"], alpha=0.5)
    plt.xlabel(feature.title())
    plt.ylabel("Preço")
    plt.title(f"Preço vs {feature.title()}")

plt.tight_layout()
plt.show()

In [ ]:
# Matriz de correlação
plt.figure(figsize=(10, 8))
correlation_matrix = df.corr()
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", center=0)
plt.title("Matriz de Correlação")
plt.show()

### Etapa 3: Preparação dos Dados


In [ ]:
# Verificar valores ausentes
print("Valores ausentes por coluna:")
print(df.isnull().sum())

# Verificar outliers
print("\nOutliers detectados (usando IQR):")
for col in df.select_dtypes(include=[np.number]).columns:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    print(f"{col}: {len(outliers)} outliers")

In [ ]:
# Separar features e target
X = df.drop("price", axis=1)
y = df["price"]

# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Dados de treino: {X_train.shape}")
print(f"Dados de teste: {X_test.shape}")

In [ ]:
# Normalização dos dados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Converter de volta para DataFrame para manter nomes das colunas
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

print("Dados normalizados com sucesso!")
print("\nMédias após normalização (devem ser ~0):")
print(X_train_scaled.mean())
print("\nDesvios padrão após normalização (devem ser ~1):")
print(X_train_scaled.std())

### Etapa 4: Seleção e Treinamento do Modelo


In [ ]:
# Treinar modelo de regressão linear
model = LinearRegression()
model.fit(X_train_scaled, y_train)

print("Modelo treinado com sucesso!")
print(f"\nCoeficientes do modelo:")
for feature, coef in zip(X_train.columns, model.coef_):
    print(f"{feature}: {coef:.3f}")
print(f"Intercept: {model.intercept_:.3f}")

### Etapa 5: Avaliação do Modelo


In [ ]:
# Fazer previsões
y_train_pred = model.predict(X_train_scaled)
y_test_pred = model.predict(X_test_scaled)

# Calcular métricas
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

print("=== Métricas de Avaliação ===")
print(f"RMSE Treino: {train_rmse:.3f}")
print(f"RMSE Teste: {test_rmse:.3f}")
print(f"R² Treino: {train_r2:.3f}")
print(f"R² Teste: {test_r2:.3f}")

# Verificar overfitting
if abs(train_rmse - test_rmse) > train_rmse * 0.1:
    print("\n⚠️ Possível overfitting detectado!")
else:
    print("\n✅ Modelo parece bem generalizado.")

In [ ]:
# Visualizar resultados
plt.figure(figsize=(15, 5))

# Predito vs Real - Treino
plt.subplot(1, 3, 1)
plt.scatter(y_train, y_train_pred, alpha=0.6)
plt.plot([y_train.min(), y_train.max()], [y_train.min(), y_train.max()], "r--", lw=2)
plt.xlabel("Valores Reais")
plt.ylabel("Valores Preditos")
plt.title(f"Treino - R²: {train_r2:.3f}")

# Predito vs Real - Teste
plt.subplot(1, 3, 2)
plt.scatter(y_test, y_test_pred, alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], "r--", lw=2)
plt.xlabel("Valores Reais")
plt.ylabel("Valores Preditos")
plt.title(f"Teste - R²: {test_r2:.3f}")

# Resíduos
plt.subplot(1, 3, 3)
residuals = y_test - y_test_pred
plt.scatter(y_test_pred, residuals, alpha=0.6)
plt.axhline(y=0, color="r", linestyle="--")
plt.xlabel("Valores Preditos")
plt.ylabel("Resíduos")
plt.title("Análise de Resíduos")

plt.tight_layout()
plt.show()

## 3. Ética em Dados e ML

### Princípios importantes:

1. **Privacidade**: Proteger dados pessoais
2. **Transparência**: Explicar como o modelo funciona
3. **Fairness**: Evitar discriminação e viés
4. **Responsabilidade**: Assumir responsabilidade pelos resultados
5. **Consentimento**: Obter permissão para uso dos dados

### Exemplos de problemas éticos:

- Algoritmos de contratação que discriminam por gênero
- Sistemas de reconhecimento facial com viés racial
- Uso de dados sem consentimento
- Falta de transparência em decisões automatizadas


## 4. Boas Práticas

### Durante o projeto:

- **Documentar** todas as decisões e experimentos
- **Versionar** código e dados
- **Validar** resultados com especialistas do domínio
- **Testar** o modelo em dados nunca vistos
- **Monitorar** performance em produção

### Checklist de qualidade:

- ✅ Problema bem definido
- ✅ Dados explorados e limpos
- ✅ Modelo avaliado adequadamente
- ✅ Métricas apropriadas para o problema
- ✅ Considerações éticas avaliadas
- ✅ Código reprodutível


## 5. Mini-Quiz

**Pergunta 1:** Qual é a primeira etapa de um projeto ML?

- a) Coleta de dados
- b) Definição do problema
- c) Treinamento do modelo

**Pergunta 2:** Por que dividimos os dados em treino e teste?

- a) Para acelerar o treinamento
- b) Para economizar memória
- c) Para avaliar a capacidade de generalização

**Pergunta 3:** O que indica um possível overfitting?

- a) Alta performance em treino e teste
- b) Baixa performance em treino e teste
- c) Alta performance em treino, baixa em teste


## Respostas do Quiz

1. **b) Definição do problema** - Precisamos saber o que queremos resolver
2. **c) Para avaliar a capacidade de generalização** - Teste simula dados nunca vistos
3. **c) Alta performance em treino, baixa em teste** - Modelo decorou os dados de treino


## Próximos Passos

Na próxima lição, começaremos a estudar:

- Regressão Linear em detalhes
- Diferentes tipos de regressão
- Técnicas de regularização
- Métricas específicas para regressão
